In [2]:
import pandas as pd
import numpy as np

In [3]:
# Read in T1_manipulated df, casas df + estudio_trabajo df
T0 = pd.read_csv('/Users/katherinewu/Library/CloudStorage/GoogleDrive-kath.j.wu@gmail.com/My Drive/5. Coding/na-data-exploration/TrUST Data/T0_manipulated.csv')
T1 = pd.read_csv('/Users/katherinewu/Library/CloudStorage/GoogleDrive-kath.j.wu@gmail.com/My Drive/5. Coding/na-data-exploration/TrUST Data/T1_manipulated.csv')
T2 = pd.read_csv('/Users/katherinewu/Library/CloudStorage/GoogleDrive-kath.j.wu@gmail.com/My Drive/5. Coding/na-data-exploration/TrUST Data/T2_manipulated.csv')
T3 = pd.read_csv('/Users/katherinewu/Library/CloudStorage/GoogleDrive-kath.j.wu@gmail.com/My Drive/5. Coding/na-data-exploration/TrUST Data/T3_manipulated.csv')
casas = pd.read_csv('/Users/katherinewu/Library/CloudStorage/GoogleDrive-kath.j.wu@gmail.com/My Drive/5. Coding/na-data-exploration/TrUST Data/casa.csv')
trabajo = pd.read_csv('/Users/katherinewu/Library/CloudStorage/GoogleDrive-kath.j.wu@gmail.com/My Drive/5. Coding/na-data-exploration/TrUST Data/estudio_trabajo.csv')

/var/folders/3s/g6b4txxj00d2fwkzq_fb1sbw0000gn/T/ipykernel_67413/2688863755.py:2: DtypeWarning: Columns (18,20,27,83,85,87,92,97,102,146,148,150,152,154,157,177,204,206,208,216,224,229,232,237,240,329,336,343,365,388,416,421,422,423,424,425,426,479,495,511,519,524,538,539,541) have mixed types. Specify dtype option on import or set low_memory=False.
  T0 = pd.read_csv('/Users/katherinewu/Library/CloudStorage/GoogleDrive-kath.j.wu@gmail.com/My Drive/5. Coding/na-data-exploration/TrUST Data/T0_manipulated.csv')
/var/folders/3s/g6b4txxj00d2fwkzq_fb1sbw0000gn/T/ipykernel_67413/2688863755.py:3: DtypeWarning: Columns (58,114,121,124,131,194,196,335,397,404,405,450,541,581,597,598,605,607,611) have mixed types. Specify dtype option on import or set low_memory=False.
  T1 = pd.read_csv('/Users/katherinewu/Library/CloudStorage/GoogleDrive-kath.j.wu@gmail.com/My Drive/5. Coding/na-data-exploration/TrUST Data/T1_manipulated.csv')
/var/folders/3s/g6b4txxj00d2fwkzq_fb1sbw0000gn/T/ipykernel_67413/26

In [4]:
# Check that df's were read in correctly
#T0.head()
#T1.head()
#casas.head() # transmi_4 column = addresses
#trabajo.head()

In [5]:
# Select only participants from Ciudad Bolivar in T0 + check length of df
T0_CB = T0[T0['localidad'] == 'Ciudad Bolivar']
T0_CB.head()
print(f"Total number of rows: {len(T0_CB)}") # Based on 'TrUST_T3 Sampling Strategy version larga', n = 1.031
print(f"Number of columns: {len(T0_CB.columns)}") # # of columns in 'Diccionario Encuesta Hogares' for T0-transmicable = 549


Total number of rows: 1031
Number of columns: 553


In [6]:
# Select only participants from Ciudad Bolivar in casas + check length of df
casas_CB = casas[casas['transmi__3'] == 'Ciudad Bolivar']
casas_CB.head()
print(f"Total number of rows: {len(casas_CB)}") # This # matches T0
# Based on data exploration, n = 2052 for CB + SC

Total number of rows: 1028


In [7]:
# Select only participants from Ciudad Bolivar in T1 + check length of df
T1.head()
T1_CB = T1[T1['T1_localidad'] == 'Ciudad Bolivar'] 
print(f"Total number of rows: {len(T1_CB)}") # This # matches 'TrUST_T3 Sampling Strategy version larga', n = 825

Total number of rows: 825


In [8]:
# Select only participants from Ciudad Bolivar in T2 + check length of df
T2.head()
T2_CB = T2[T2['T2_localidad'] == 1] # According 'Diccionario Encuesta Hogares' to 1 = CB 
#T2_CB = T2[T2['T2_Q3dir'] == 1] # Wanted to add question confirming that address stayed the same, but ended up filtering the df in a way that didn't match the sampling strategy #s
print(f"Total number of rows: {len(T2_CB)}") # this # matches 'TrUST_T3 Sampling Strategy version larga', n = 694

Total number of rows: 694


In [9]:
# Select only participants from Ciudad Bolivar in T3 + check length of df
T3.head()
T3_CB = T3[T3['T3_ZONA_BASE'] == 'Ciudad Bolivar'] # According 'Diccionario Encuesta Hogares' to 1 = CB 
#T3_CB = T3[T3['T3_P3'] == 1] # Wanted to add question confirming that address stayed the same, but ended up filtering the df in a way that didn't match the sampling strategy #s
# Also tried to use 'T3_LOCALIDAD' == 19 for CB, but the length of the df also didn't match the sampling strategy #s
print(f"Total number of rows: {len(T3_CB)}") # this # matches 'TrUST_T3 Sampling Strategy version larga', n = 611

Total number of rows: 611


In [10]:
# Test merging casas df with trabajo df to see how many participants we can match
c_t = pd.merge(casas, trabajo, left_on='transmi__1', right_on='Name', how='inner')
print(c_t.head())
print(f"Total number of rows: {len(c_t)}") # all participants matched

# Select columns of interest in new casas + trabajo df
c_t_select = c_t[['transmi__4', 'transmi__1', 'lat_x', 'lon_x', 'Name', 'lat_y', 'lon_y']]
c_t_select.rename(columns={'transmi__4': 'Q2dir','transmi__1': 'casas_ID', 'Name': 'trabajo_ID'}, inplace=True)
c_t_select.head()

   FID_x  transmi__1  transmi__2      transmi__3            transmi__4  \
0      1           2   204106017  Ciudad Bolivar   KR 27 I 72 C SUR 49   
1      4           5   204106025  Ciudad Bolivar      KR 27 L 7331 SUR   
2      5           6   204106025  Ciudad Bolivar      KR 27 K 7344 SUR   
3      8           9   204106066  Ciudad Bolivar   CL 71 T SUR 27 J 04   
4     13          14     1305009   San Cristobal  CL 36 F SUR 218 ESTE   

   Manzanas_1  Manzanas_5  Manzanas_6   Manzanas_7   Manzanas_8  ...  \
0    20410617   204106017           1  990509.1141  994604.8881  ...   
1    20410625   204106025           1  990451.6518  994621.6796  ...   
2    20410625   204106025           1  990451.6518  994621.6796  ...   
3    20410666   204106066           0  990480.4592  994613.4925  ...   
4      130509     1305009           2  998270.2956  995760.6309  ...   

   park_densi  parques2     lat_x      lon_x  FID_y  OBJECTID  Name  \
0    0.039610  3.960956  4.547407 -74.163033      0

/var/folders/3s/g6b4txxj00d2fwkzq_fb1sbw0000gn/T/ipykernel_67413/1968100874.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  c_t_select.rename(columns={'transmi__4': 'Q2dir','transmi__1': 'casas_ID', 'Name': 'trabajo_ID'}, inplace=True)


,Q2dir,casas_ID,lat_x,lon_x,trabajo_ID,lat_y,lon_y
0,KR 27 I 72 C SUR 49,2,4.547407,-74.163033,2,4.759885,-74.064452
1,KR 27 L 7331 SUR,5,4.547558,-74.163551,5,4.609500,-74.095503
2,KR 27 K 7344 SUR,6,4.547558,-74.163551,6,4.537814,-74.161777
3,CL 71 T SUR 27 J 04,9,4.547484,-74.163292,9,4.671032,-74.155843
4,CL 36 F SUR 218 ESTE,14,4.557863,-74.093095,14,4.551624,-74.082405


In [11]:
# Now that I've confirmed that the household survey #s for all time points (T0, T1, T2 + T3) closely align with 'TrUST_T3 Sampling Strategy version larga', I'll begin 
    # to merge them with the lat/long from the casas df - I need to use T0 bc it's the only df with addresses included (Q2dir), which is what I'll merge on
'''T0_c_merged = pd.merge(T0_CB, c_t, left_on='Q2dir', right_on='transmi__4', how='inner')
print(T0_c_merged.head())
print(f"Total number of rows: {len(T0_c_merged)}") # 878 vs. 877 when done with the entire df not isolated for CB locality'''


'T0_c_merged = pd.merge(T0_CB, c_t, left_on=\'Q2dir\', right_on=\'transmi__4\', how=\'inner\')\nprint(T0_c_merged.head())\nprint(f"Total number of rows: {len(T0_c_merged)}") # 878 vs. 877 when done with the entire df not isolated for CB locality'

In [12]:
# Now that I've confirmed that the household survey #s for all time points (T0, T1, T2 + T3) closely align with 'TrUST_T3 Sampling Strategy version larga', I'll begin 
    # to merge them with the lat/long from the casas df - I need to use T0 bc it's the only df with addresses included (Q2dir), which is what I'll merge on
'''T0_c_merged = pd.merge(T0_CB, casas_CB, left_on='Q2dir', right_on='transmi__4', how='inner')
print(T0_c_merged.head())
print(f"Total number of rows: {len(T0_c_merged)}") # 878 vs. 877 when done with the entire df not isolated for CB locality'''


'T0_c_merged = pd.merge(T0_CB, casas_CB, left_on=\'Q2dir\', right_on=\'transmi__4\', how=\'inner\')\nprint(T0_c_merged.head())\nprint(f"Total number of rows: {len(T0_c_merged)}") # 878 vs. 877 when done with the entire df not isolated for CB locality'

In [13]:
# Isolate df down to the most basic items of interest to test #s
T0_c = T0_c_merged[['ID', 'transmi__1', 'localidad', 'transmi__3', 'Q2dir', 'transmi__4', 'lat', 'lon', 'Q2dir', 'Q11', 'Q12', 'Q13', 'Q15', 'Q16', 'Q19', 'Q87c1', 'Q115', 'Q116']]

NameError: name 'T0_c_merged' is not defined

In [ ]:
T0_c.head()

,ID,transmi__1,localidad,transmi__3,Q2dir,transmi__4,lat,lon,Q2dir,Q11,Q12,Q13,Q15,Q16,Q19,Q87c1,Q115,Q116
0,ID15490250091142,1400,Ciudad Bolivar,Ciudad Bolivar,CL 69 I BIS SUR 18 M 45,CL 69 I BIS SUR 18 M 45,4.553791,-74.153024,CL 69 I BIS SUR 18 M 45,2,22,4,1,5,888,1,2,2
1,ID15210400150200,1232,Ciudad Bolivar,Ciudad Bolivar,CL 62 A SUR 18 Q 53,CL 62 A SUR 18 Q 53,4.558492,-74.141770,CL 62 A SUR 18 Q 53,1,50,1,1,4,888,888,2,1
2,ID15800280212297,2004,Ciudad Bolivar,Ciudad Bolivar,CL 73 C BIS SUR 27 55,CL 73 C BIS SUR 27 55,4.544794,-74.161377,CL 73 C BIS SUR 27 55,2,24,1,1,3,888,888,1,1
3,ID15800380210206,2022,Ciudad Bolivar,Ciudad Bolivar,KR 27 BIS 72 C 51 SUR,KR 27 BIS 72 C 51 SUR,4.546388,-74.160283,KR 27 BIS 72 C 51 SUR,2,73,1,1,3,888,888,1,1
4,ID15800390120204,2030,Ciudad Bolivar,Ciudad Bolivar,KR 27 BIS 72 C 16 SUR,KR 27 BIS 72 C 16 SUR,4.546293,-74.160007,KR 27 BIS 72 C 16 SUR,2,39,4,1,4,888,888,2,2


In [ ]:
T3_CB_select = T3_CB[['T3_ID_INF', 'T3_ZONA_BASE']]
T3_CB_select.head()

,T3_ID_INF,T3_ZONA_BASE
0,ID15530030362396,Ciudad Bolivar
6,ID15520030032175,Ciudad Bolivar
9,ID15520860032491,Ciudad Bolivar
10,ID15520860422494,Ciudad Bolivar
11,ID15530600120190,Ciudad Bolivar


In [ ]:
# Test merging T0_c merged df on T3 to see how many participants we can match in the final timeframe
T03_c = pd.merge(T0_c, T3_CB_select, left_on='ID', right_on='T3_ID_INF', how='inner')
print(T03_c.head())
print(f"Total number of rows: {len(T03_c)}") # sample size in sampling strategy is n = 598

                 ID  transmi__1       localidad      transmi__3  \
0  ID15490250091142        1400  Ciudad Bolivar  Ciudad Bolivar   
1  ID15210400150200        1232  Ciudad Bolivar  Ciudad Bolivar   
2  ID15800280212297        2004  Ciudad Bolivar  Ciudad Bolivar   
3  ID15800380210206        2022  Ciudad Bolivar  Ciudad Bolivar   
4  ID15520940752473        1550  Ciudad Bolivar  Ciudad Bolivar   

                     Q2dir               transmi__4       lat        lon  \
0  CL 69 I BIS SUR 18 M 45  CL 69 I BIS SUR 18 M 45  4.553791 -74.153024   
1      CL 62 A SUR 18 Q 53      CL 62 A SUR 18 Q 53  4.558492 -74.141770   
2    CL 73 C BIS SUR 27 55    CL 73 C BIS SUR 27 55  4.544794 -74.161377   
3    KR 27 BIS 72 C 51 SUR    KR 27 BIS 72 C 51 SUR  4.546388 -74.160283   
4      TV 18 P 69 A 33 SUR      TV 18 P 69 A 33 SUR  4.554053 -74.149809   

                     Q2dir  Q11  Q12  Q13  Q15  Q16  Q19  Q87c1  Q115  Q116  \
0  CL 69 I BIS SUR 18 M 45    2   22    4    1    5  888     

In [ ]:
T2_CB.head()

,T2_ID,T2_localidad,T2_Q3dir,T2_Q4_10,T2_Q4_8,T2_Q4_2,T2_Q4_3,T2_Q4_4,T2_Q5_1,T2_Q5_2,...,T2_123_7,T2_123_8,T2_123_8_TEXT,T2_124,T2_Q125,T2_Q126,T2_Q127,T2_Q128,T2_Q129,T2_Q130
0,ID15150290512124,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,NaN,NaN,2,1,1.0,2,2,1,5/02/1978
1,ID15150150031132,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2,1,3.0,2,1,2,19/10/1978
3,ID15150300062127,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2,1,3.0,2,1,2,25/10/1969
4,ID15150270151137,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,3,1,3.0,1,1,2,12/10/1999
5,ID15150310362117,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2,1,1.0,2,1,2,9/05/1944


In [ ]:
T2_CB_select = T2_CB[['T2_ID', 'T2_localidad']]
T2_CB_select.head()

,T2_ID,T2_localidad
0,ID15150290512124,1
1,ID15150150031132,1
3,ID15150300062127,1
4,ID15150270151137,1
5,ID15150310362117,1


In [ ]:
# Test merging T03_c merged df with T2 to see how many participants we can match
T023_c = pd.merge(T03_c, T2_CB_select, left_on='ID', right_on='T2_ID', how='inner')
print(T023_c.head())
print(f"Total number of rows: {len(T023_c)}")

                 ID  transmi__1       localidad      transmi__3  \
0  ID15490250091142        1400  Ciudad Bolivar  Ciudad Bolivar   
1  ID15210400150200        1232  Ciudad Bolivar  Ciudad Bolivar   
2  ID15800280212297        2004  Ciudad Bolivar  Ciudad Bolivar   
3  ID15800380210206        2022  Ciudad Bolivar  Ciudad Bolivar   
4  ID15520940752473        1550  Ciudad Bolivar  Ciudad Bolivar   

                     Q2dir               transmi__4       lat        lon  \
0  CL 69 I BIS SUR 18 M 45  CL 69 I BIS SUR 18 M 45  4.553791 -74.153024   
1      CL 62 A SUR 18 Q 53      CL 62 A SUR 18 Q 53  4.558492 -74.141770   
2    CL 73 C BIS SUR 27 55    CL 73 C BIS SUR 27 55  4.544794 -74.161377   
3    KR 27 BIS 72 C 51 SUR    KR 27 BIS 72 C 51 SUR  4.546388 -74.160283   
4      TV 18 P 69 A 33 SUR      TV 18 P 69 A 33 SUR  4.554053 -74.149809   

                     Q2dir  Q11  ...  Q15  Q16  Q19  Q87c1  Q115  Q116  \
0  CL 69 I BIS SUR 18 M 45    2  ...    1    5  888      1     2  

In [ ]:
T1_CB.head() # look at ID11060170121129 -- address changed but remained within CB, I'm most likely missing a few of these + need to decide whether I keep only the participants that
    # remained in the same location for all 4 time periods

,T1_ID,T1_localidad,T1_Q2dir,T1_Q3dir,T1_Q3barrio,T1_Q3localidad,T1_Q3ciudad,T1_Q3pais,T1_Q4_1,T1_Q4_2,...,T1_direccionestudio,T1_Intersection_density,T1_park_density_area,T1_avg_slope,T1_Dist_trabajo,T1_Dist_Camina_Trabajo,T1_Dist_Estacion,T1_Dist_Portal,T1_Dist_Parque,T1_Dist_Equip
853,ID11060170121129,Ciudad Bolivar,2,TV 18 P BIS # 56 T - 59 SUR,Villas del diamante,Ciudad Bolivar,Bogota,Colombia,2,2,...,NaN,721.694105,0.058923,19.891395,4366.00356,249.0,174.663560,1821.764705,17.638242,100.503399
854,ID11060170270211,Ciudad Bolivar,1,888,888,888,888,888,888,888,...,NaN,489.534771,0.039610,4.600955,NaN,NaN,280.872341,4895.835533,91.306598,500.683902
855,ID11060230092012,Ciudad Bolivar,1,888,888,888,888,888,888,888,...,NaN,524.670018,0.087167,18.393585,NaN,NaN,189.324835,3994.809449,18.313958,115.879924
856,ID11060250012108,Ciudad Bolivar,1,888,888,888,888,888,888,888,...,NaN,462.240360,0.037207,4.105512,NaN,NaN,312.213538,4964.090602,146.589566,477.486163
857,ID11060250182043,Ciudad Bolivar,1,888,888,888,888,888,888,888,...,NaN,462.240360,0.037207,4.105512,NaN,NaN,312.213538,4964.090602,146.589566,477.486163


In [ ]:
T1_CB_select = T1_CB[['T1_ID', 'T1_localidad', 'T1_Q2dir']]
T1_CB_select.head()

,T1_ID,T1_localidad,T1_Q2dir
853,ID11060170121129,Ciudad Bolivar,2
854,ID11060170270211,Ciudad Bolivar,1
855,ID11060230092012,Ciudad Bolivar,1
856,ID11060250012108,Ciudad Bolivar,1
857,ID11060250182043,Ciudad Bolivar,1


In [ ]:
# Test merging T023_c merged df with T1 to see how many participants we can match
T0123_c = pd.merge(T023_c, T1_CB_select, left_on='ID', right_on='T1_ID', how='inner')
print(T0123_c.head())
print(f"Total number of rows: {len(T0123_c)}") # n = 515, no loss between T023_c + T0123_c

                 ID  transmi__1       localidad      transmi__3  \
0  ID15490250091142        1400  Ciudad Bolivar  Ciudad Bolivar   
1  ID15210400150200        1232  Ciudad Bolivar  Ciudad Bolivar   
2  ID15800280212297        2004  Ciudad Bolivar  Ciudad Bolivar   
3  ID15800380210206        2022  Ciudad Bolivar  Ciudad Bolivar   
4  ID15520940752473        1550  Ciudad Bolivar  Ciudad Bolivar   

                     Q2dir               transmi__4       lat        lon  \
0  CL 69 I BIS SUR 18 M 45  CL 69 I BIS SUR 18 M 45  4.553791 -74.153024   
1      CL 62 A SUR 18 Q 53      CL 62 A SUR 18 Q 53  4.558492 -74.141770   
2    CL 73 C BIS SUR 27 55    CL 73 C BIS SUR 27 55  4.544794 -74.161377   
3    KR 27 BIS 72 C 51 SUR    KR 27 BIS 72 C 51 SUR  4.546388 -74.160283   
4      TV 18 P 69 A 33 SUR      TV 18 P 69 A 33 SUR  4.554053 -74.149809   

                     Q2dir  Q11  ...  Q87c1  Q115  Q116         T3_ID_INF  \
0  CL 69 I BIS SUR 18 M 45    2  ...      1     2     2  ID1549

In [ ]:
# Save T0123_c df to csv to ingest into QGIS
#T0123_c.to_csv('/Users/katherinewu/Library/CloudStorage/GoogleDrive-kath.j.wu@gmail.com/My Drive/5. Coding/na-data-exploration/TrUST Data/T0123_c.csv', index=False)

In [ ]:
print(trabajo.head())

   FID  OBJECTID  Name                             FolderPath       lat  \
0    0         1     2  Georreferenciación/Georreferenciación  4.759885   
1    1         2     6  Georreferenciación/Georreferenciación  4.537814   
2    2         3     5  Georreferenciación/Georreferenciación  4.609500   
3    3         4    22  Georreferenciación/Georreferenciación  4.552240   
4    4         5     9  Georreferenciación/Georreferenciación  4.671032   

         lon  
0 -74.064452  
1 -74.161777  
2 -74.095503  
3 -74.092320  
4 -74.155843  


In [ ]:
# Change type of ID columns for casas + trabajo to integer to merge
casas['transmi__1'] = casas['transmi__1'].astype(int)
trabajo['Name'] = trabajo['Name'].astype(int)

In [ ]:
# Test merging casas df with trabajo df to see how many participants we can match
c_t = pd.merge(casas, trabajo, left_on='transmi__1', right_on='Name', how='inner')
print(c_t.head())
print(f"Total number of rows: {len(c_t)}") # all participants matched

   FID_x  transmi__1  transmi__2      transmi__3            transmi__4  \
0      1           2   204106017  Ciudad Bolivar   KR 27 I 72 C SUR 49   
1      4           5   204106025  Ciudad Bolivar      KR 27 L 7331 SUR   
2      5           6   204106025  Ciudad Bolivar      KR 27 K 7344 SUR   
3      8           9   204106066  Ciudad Bolivar   CL 71 T SUR 27 J 04   
4     13          14     1305009   San Cristobal  CL 36 F SUR 218 ESTE   

   Manzanas_1  Manzanas_5  Manzanas_6   Manzanas_7   Manzanas_8  ...  \
0    20410617   204106017           1  990509.1141  994604.8881  ...   
1    20410625   204106025           1  990451.6518  994621.6796  ...   
2    20410625   204106025           1  990451.6518  994621.6796  ...   
3    20410666   204106066           0  990480.4592  994613.4925  ...   
4      130509     1305009           2  998270.2956  995760.6309  ...   

   park_densi  parques2     lat_x      lon_x  FID_y  OBJECTID  Name  \
0    0.039610  3.960956  4.547407 -74.163033      0

In [ ]:
# Test merging casas df with trabajo df to see how many participants we can match
c_t = pd.merge(casas, trabajo, left_on='transmi__1', right_on='Name', how='inner')
print(c_t.head())
print(f"Total number of rows: {len(c_t)}") # all participants matched

# Select columns of interest in new casas + trabajo df
c_t_select = c_t[['transmi__1', 'lat_x', 'lon_x', 'Name', 'lat_y', 'lon_y']]
c_t_select.rename(columns={'transmi__1': 'casas_ID', 'Name': 'trabajo_ID'}, inplace=True)
c_t_select.head()

/var/folders/3s/g6b4txxj00d2fwkzq_fb1sbw0000gn/T/ipykernel_31997/3235638919.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  c_t_select.rename(columns={'transmi__1': 'casas_ID', 'Name': 'trabajo_ID'}, inplace=True)


,casas_ID,lat_x,lon_x,trabajo_ID,lat_y,lon_y
0,2,4.547407,-74.163033,2,4.759885,-74.064452
1,5,4.547558,-74.163551,5,4.609500,-74.095503
2,6,4.547558,-74.163551,6,4.537814,-74.161777
3,9,4.547484,-74.163292,9,4.671032,-74.155843
4,14,4.557863,-74.093095,14,4.551624,-74.082405


In [ ]:
T0123_c['transmi__1'] = T0123_c['transmi__1'].astype(int)
c_t_select.loc[:, 'casas_ID'] = c_t_select['casas_ID'].astype(int)

/var/folders/3s/g6b4txxj00d2fwkzq_fb1sbw0000gn/T/ipykernel_31997/3463632722.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  c_t_select.loc[:, 'casas_ID'] = c_t_select['casas_ID'].astype(int)


In [ ]:
# Will now combine T0123 df wih trabajo date to add in each participant's work location
T0123_ct = pd.merge(T0123_c, c_t, left_on='transmi__1', right_on='transmi__1', how='inner') # Refer to email from MAW on dataset details
T0123_ct.head()
print(f"Total number of rows: {len(T0123_ct)}") # all participants matched

Total number of rows: 150


In [ ]:
T0123_ct

,ID,transmi__1,localidad,transmi__3,Q2dir,transmi__4,lat,lon,Q2dir,Q11,...,T2_localidad,T1_ID,T1_localidad,T1_Q2dir,casas_ID,lat_x,lon_x,trabajo_ID,lat_y,lon_y
0,ID15490250091142,1400,Ciudad Bolivar,Ciudad Bolivar,CL 69 I BIS SUR 18 M 45,CL 69 I BIS SUR 18 M 45,4.553791,-74.153024,CL 69 I BIS SUR 18 M 45,2,...,1,ID15490250091142,Ciudad Bolivar,2,1400,4.553791,-74.153024,1400,4.653890,-74.068424
1,ID15530250061418,1613,Ciudad Bolivar,Ciudad Bolivar,KR 18 N BIS A 66 D 11,KR 18 N BIS A 66 D 11,4.554223,-74.146050,KR 18 N BIS A 66 D 11,2,...,1,ID15530250061418,Ciudad Bolivar,1,1613,4.554223,-74.146050,1613,4.763943,-74.032297
2,ID15530170031401,1607,Ciudad Bolivar,Ciudad Bolivar,KR 18 Q 66 B 23 SUR,KR 18 Q 66 B 23 SUR,4.555241,-74.146830,KR 18 Q 66 B 23 SUR,2,...,1,ID15530170031401,Ciudad Bolivar,1,1607,4.555241,-74.146830,1607,4.611747,-74.162797
3,ID15800350451306,2012,Ciudad Bolivar,Ciudad Bolivar,KR 27 A 73 54 SUR,KR 27 A 73 54 SUR,4.544984,-74.160630,KR 27 A 73 54 SUR,1,...,1,ID15800350451306,Ciudad Bolivar,1,2012,4.544984,-74.160630,2012,4.671269,-74.114723
4,ID15490850211060,1444,Ciudad Bolivar,Ciudad Bolivar,CL 69 T SUR 18 M 63,CL 69 T SUR 18 M 63,4.550668,-74.152273,CL 69 T SUR 18 M 63,2,...,1,ID15490850211060,Ciudad Bolivar,1,1444,4.550668,-74.152273,1444,4.728757,-74.049864
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,ID15020420092440,1072,Ciudad Bolivar,Ciudad Bolivar,DG 64 SUR 19 41,DG 64 SUR 19 41,4.558842,-74.144767,DG 64 SUR 19 41,2,...,1,ID15020420092440,Ciudad Bolivar,1,1072,4.558842,-74.144767,1072,4.610978,-74.091754
146,ID15240220271110,1668,Ciudad Bolivar,Ciudad Bolivar,CL 65 SUR 18 N 84,CL 65 SUR 18 N 84,4.556008,-74.143836,CL 65 SUR 18 N 84,2,...,1,ID15240220271110,Ciudad Bolivar,1,1668,4.556008,-74.143836,1668,4.605262,-74.068210
147,ID15520810181091,1523,Ciudad Bolivar,Ciudad Bolivar,KR 18 R 69 35 SUR,KR 18 R 69 35 SUR,4.555161,-74.149019,KR 18 R 69 35 SUR,1,...,1,ID15520810181091,Ciudad Bolivar,1,1523,4.555161,-74.149019,1523,4.698260,-74.030176
148,ID15020260061447,1058,Ciudad Bolivar,Ciudad Bolivar,DG 64 BIS A SUR 19 A 19,DG 64 BIS A SUR 19 A 19,4.558892,-74.145342,DG 64 BIS A SUR 19 A 19,1,...,1,ID15020260061447,Ciudad Bolivar,1,1058,4.558892,-74.145342,1058,4.574028,-74.154622


In [ ]:
T0123_ct.head()
len(T0123_ct)

150

In [ ]:
len(T0123_c)

515

In [ ]:
column_values_ct = T0123_ct['transmi__1'].tolist()
print(column_values_ct)

[1400, 1613, 1607, 2012, 1444, 1911, 1859, 2039, 1503, 1995, 1882, 1993, 1391, 2021, 1355, 1406, 1924, 1758, 1574, 1597, 1350, 1726, 1773, 1774, 1719, 1925, 1738, 1270, 1590, 2000, 1641, 1642, 1736, 1319, 1327, 2003, 1797, 1519, 1555, 1623, 1114, 1306, 1829, 1767, 1768, 1285, 1430, 1432, 1823, 1552, 1732, 1772, 1280, 1831, 1268, 1267, 1334, 1784, 1312, 1380, 1341, 1426, 1655, 1124, 1039, 1661, 1335, 1449, 1447, 1240, 1584, 1561, 1318, 1671, 2027, 1686, 1122, 1712, 1472, 1669, 1625, 1558, 1878, 1943, 1906, 1969, 1970, 1854, 1870, 1910, 1871, 1959, 1971, 1839, 1926, 1927, 1904, 1979, 2011, 1877, 1846, 1905, 1907, 1950, 2047, 1978, 1919, 1869, 1929, 2014, 2015, 1947, 1948, 1747, 1864, 1956, 1862, 1837, 2031, 1162, 1237, 1490, 1801, 1296, 1721, 1734, 1323, 1556, 1658, 1222, 1680, 1533, 1517, 1518, 1042, 1620, 1717, 1598, 1622, 1674, 1567, 1461, 1710, 1068, 1219, 1072, 1668, 1523, 1058, 1866]


In [ ]:
column_values_c = T0123_c['transmi__1'].tolist()
print(column_values_c)

[1400, 1232, 2004, 2022, 1550, 1115, 1433, 1479, 1613, 1607, 1679, 2012, 1077, 1521, 1512, 1826, 1444, 1911, 1859, 2039, 1377, 1282, 1503, 1506, 1498, 1450, 1502, 1425, 1437, 1393, 1251, 1995, 1882, 1928, 1993, 1391, 1998, 1742, 1986, 1274, 1278, 2021, 1355, 1406, 1402, 1924, 1758, 1572, 1574, 1608, 1301, 1597, 1299, 1603, 1409, 1330, 1333, 1351, 1610, 1350, 1726, 1773, 1774, 1798, 1302, 1778, 1779, 1780, 1719, 1413, 1411, 1925, 1357, 1259, 1441, 1815, 1248, 1271, 1738, 1270, 1410, 1739, 1297, 1491, 1253, 1493, 10, 12, 2002, 1590, 2000, 1637, 1632, 1641, 1642, 1636, 1640, 1736, 1319, 1899, 1438, 1729, 1390, 1903, 1996, 1638, 1327, 1322, 2003, 1591, 1720, 1797, 1796, 1722, 1519, 1555, 1465, 1585, 1623, 1389, 1582, 1614, 1093, 1114, 1101, 1725, 1309, 1306, 1381, 1829, 1382, 1767, 1768, 1285, 1430, 1432, 1827, 1823, 1053, 1035, 1552, 1987, 1763, 1645, 1041, 1732, 1250, 1225, 1529, 1530, 1456, 1455, 1514, 1535, 1181, 1363, 1429, 1772, 1718, 1111, 1276, 1275, 1280, 1281, 1359, 2020, 1075, 1

In [ ]:
column_values_ct_sel = c_t_select['casas_ID'].tolist()
print(column_values_ct_sel)

[2, 5, 6, 9, 14, 20, 22, 23, 24, 25, 29, 30, 35, 36, 39, 41, 42, 47, 51, 52, 54, 57, 60, 61, 64, 65, 67, 73, 74, 75, 76, 82, 83, 92, 96, 97, 102, 110, 118, 126, 133, 135, 137, 138, 142, 150, 152, 158, 162, 164, 167, 170, 171, 182, 186, 190, 192, 194, 197, 199, 210, 211, 213, 214, 215, 218, 219, 221, 225, 226, 235, 240, 242, 244, 245, 247, 248, 249, 261, 266, 267, 268, 271, 272, 275, 279, 280, 281, 285, 286, 290, 292, 293, 295, 299, 301, 302, 303, 308, 311, 313, 317, 319, 321, 322, 325, 326, 331, 333, 341, 343, 344, 349, 350, 354, 363, 364, 366, 368, 369, 370, 372, 373, 378, 388, 390, 391, 392, 393, 396, 398, 399, 401, 405, 407, 409, 410, 411, 415, 417, 419, 424, 425, 426, 428, 429, 432, 433, 435, 436, 437, 441, 442, 447, 448, 450, 451, 453, 455, 456, 457, 458, 462, 463, 465, 472, 474, 476, 484, 485, 486, 487, 488, 492, 495, 496, 497, 498, 503, 504, 505, 510, 511, 513, 514, 518, 520, 522, 524, 525, 528, 529, 533, 540, 542, 548, 549, 552, 559, 562, 565, 568, 569, 575, 578, 579, 583, 588,

In [ ]:
# Find common values
common_values = set(c_t_select['casas_ID']).intersection(set(T0123_c['transmi__1']))

# Count the number of common values
common_count = len(common_values)

#print(f"Common values: {common_values}")
print(f"Number of common values: {common_count}")

Number of common values: 150


In [ ]:
# Find common values
common_values = set(trabajo['Name']).intersection(set(T0123_c['transmi__1']))

# Count the number of common values
common_count = len(common_values)

#print(f"Common values: {common_values}")
print(f"Number of common values: {common_count}")

Number of common values: 150


In [ ]:
# Find common values
common_values = set(trabajo['Name']).intersection(set(casas['transmi__1']))

# Count the number of common values
common_count = len(common_values)

#print(f"Common values: {common_values}")
print(f"Number of common values: {common_count}")

Number of common values: 691


In [ ]:
# Find common values
common_values = set(casas['transmi__1']).intersection(set(T0123_c['transmi__1']))

# Count the number of common values
common_count = len(common_values)

#print(f"Common values: {common_values}")
print(f"Number of common values: {common_count}")

Number of common values: 515


In [ ]:
trabajo.dtypes

FID             int64
OBJECTID        int64
Name            int64
FolderPath     object
lat           float64
lon           float64
dtype: object

In [ ]:
casas.dtypes

FID             int64
transmi__1      int64
transmi__2      int64
transmi__3     object
transmi__4     object
Manzanas_1      int64
Manzanas_5      int64
Manzanas_6      int64
Manzanas_7    float64
Manzanas_8    float64
Manzanas_9      int64
Intersecti    float64
park_densi    float64
parques2      float64
lat           float64
lon           float64
dtype: object

In [ ]:
# Convert columns to sets
set_df1 = set(T0123_c['transmi__1'])
set_df2 = set(c_t_select['casas_ID'])

# Find non-matching values from df1
non_matching_from_df1 = list(set_df1 - set_df2)

# Find non-matching values from df2
non_matching_from_df2 = list(set_df2 - set_df1)

print(f"Non-matching values from df1: {non_matching_from_df1}")
print(f"Non-matching values from df2: {non_matching_from_df2}")

Non-matching values from df1: [2050, 2052, 10, 11, 12, 1035, 1037, 1041, 1043, 1047, 1048, 1053, 1062, 1064, 1070, 1071, 1075, 1077, 1084, 1085, 1087, 1093, 1096, 1097, 1098, 1100, 1101, 1102, 1105, 1106, 1108, 1109, 1111, 1112, 1115, 1117, 1118, 1120, 1129, 1137, 1138, 1140, 1147, 1150, 1152, 1153, 1158, 1172, 1181, 1187, 1190, 1192, 1194, 1195, 1196, 1198, 1201, 1204, 1208, 1210, 1213, 1218, 1220, 1221, 1223, 1225, 1226, 1228, 1231, 1232, 1233, 1248, 1250, 1251, 1252, 1253, 1254, 1255, 1258, 1259, 1264, 1266, 1269, 1271, 1273, 1274, 1275, 1276, 1277, 1278, 1281, 1282, 1283, 1287, 1289, 1290, 1292, 1293, 1294, 1297, 1299, 1301, 1302, 1305, 1307, 1309, 1310, 1311, 1322, 1324, 1330, 1333, 1347, 1349, 1351, 1357, 1359, 1363, 1367, 1368, 1370, 1372, 1377, 1381, 1382, 1389, 1390, 1393, 1396, 1399, 1402, 1404, 1409, 1410, 1411, 1412, 1413, 1417, 1419, 1421, 1423, 1425, 1427, 1429, 1433, 1437, 1438, 1441, 1450, 1453, 1454, 1455, 1456, 1459, 1462, 1464, 1465, 1467, 1468, 1469, 1470, 1478, 147

In [ ]:
# Convert columns to sets
set_df1 = set(T0123_c['transmi__1'])
set_df2 = set(casas_CB['transmi__1'])

# Find non-matching values from df1
non_matching_from_df1 = list(set_df1 - set_df2)

# Find non-matching values from df2
non_matching_from_df2 = list(set_df2 - set_df1)

print(f"Non-matching values from df1: {non_matching_from_df1}")
print(f"Non-matching values from df2: {non_matching_from_df2}")
print(len(non_matching_from_df2))
print(len(non_matching_from_df1))

Non-matching values from df1: []
Non-matching values from df2: [2048, 1, 2, 3, 4, 5, 6, 7, 8, 9, 2049, 2051, 111, 1036, 1038, 1040, 1044, 1045, 1046, 1049, 1050, 1051, 1052, 1054, 1055, 1056, 1057, 1059, 1060, 1061, 1063, 1065, 1066, 1067, 1069, 1073, 1074, 1076, 1078, 1079, 1080, 1081, 1082, 1083, 1086, 1088, 1089, 1090, 1091, 1092, 1094, 1095, 1099, 1103, 1104, 1107, 1110, 1113, 1116, 1119, 1121, 1123, 1125, 1126, 1127, 1128, 1130, 1131, 1132, 1133, 1134, 1135, 1136, 1139, 1141, 1142, 1143, 1144, 1145, 1146, 1148, 1149, 1151, 1154, 1155, 1156, 1157, 1159, 1160, 1161, 1163, 1164, 1165, 1166, 1167, 1168, 1169, 1170, 1171, 1173, 1174, 1175, 1176, 1177, 1178, 1179, 1180, 1182, 1183, 1184, 1185, 1186, 1188, 1189, 1191, 1193, 1197, 1199, 1200, 1202, 1203, 1205, 1206, 1207, 1209, 1211, 1212, 1214, 1215, 1216, 1217, 1224, 1227, 1229, 1230, 1234, 1235, 1236, 1238, 1239, 1241, 1242, 1243, 1244, 1245, 1246, 1247, 1249, 1256, 1257, 1260, 1261, 1262, 1263, 1265, 1272, 1279, 1284, 1286, 1288, 1291

In [ ]:
print(len(casas)-len(casas_CB))
print(len(casas_CB))
print(len(casas))
print(len(casas_CB)-len(non_matching_from_df2))

1024
1028
2052
515


In [ ]:
c_t_select.head()

,Q2dir,casas_ID,lat_x,lon_x,trabajo_ID,lat_y,lon_y
0,KR 27 I 72 C SUR 49,2,4.547407,-74.163033,2,4.759885,-74.064452
1,KR 27 L 7331 SUR,5,4.547558,-74.163551,5,4.609500,-74.095503
2,KR 27 K 7344 SUR,6,4.547558,-74.163551,6,4.537814,-74.161777
3,CL 71 T SUR 27 J 04,9,4.547484,-74.163292,9,4.671032,-74.155843
4,CL 36 F SUR 218 ESTE,14,4.557863,-74.093095,14,4.551624,-74.082405


In [ ]:
print(len(casas))
casas.head()

2052


,FID,transmi__1,transmi__2,transmi__3,transmi__4,Manzanas_1,Manzanas_5,Manzanas_6,Manzanas_7,Manzanas_8,Manzanas_9,Intersecti,park_densi,parques2,lat,lon
0,0,1,204106017,Ciudad Bolivar,CL 73 SUR 27 H 21,20410617,204106017,1,990509.1141,994604.8881,204106017,488.339057,0.039513,3.951281,4.547407,-74.163033
1,1,2,204106017,Ciudad Bolivar,KR 27 I 72 C SUR 49,20410617,204106017,1,990509.1141,994604.8881,204106017,489.534771,0.039610,3.960956,4.547407,-74.163033
2,2,3,204106017,Ciudad Bolivar,KR 27 I 7338 SUR,20410617,204106017,1,990509.1141,994604.8881,204106017,489.534771,0.039610,3.960956,4.547407,-74.163033
3,3,4,204106017,Ciudad Bolivar,KR 27 I 7362 SUR,20410617,204106017,1,990509.1141,994604.8881,204106017,489.534771,0.039610,3.960956,4.547407,-74.163033
4,4,5,204106025,Ciudad Bolivar,KR 27 L 7331 SUR,20410625,204106025,1,990451.6518,994621.6796,204106025,462.240360,0.037207,3.720670,4.547558,-74.163551


In [ ]:
print(T0123_c.columns)
print(c_t_select.columns)

Index(['ID', 'transmi__1', 'localidad', 'transmi__3', 'Q2dir', 'transmi__4',
       'lat', 'lon', 'Q2dir', 'Q11', 'Q12', 'Q13', 'Q15', 'Q16', 'Q19',
       'Q87c1', 'Q115', 'Q116', 'T3_ID_INF', 'T3_ZONA_BASE', 'T2_ID',
       'T2_localidad', 'T1_ID', 'T1_localidad', 'T1_Q2dir'],
      dtype='object')
Index(['Q2dir', 'casas_ID', 'lat_x', 'lon_x', 'trabajo_ID', 'lat_y', 'lon_y'], dtype='object')


In [ ]:
T0123_c_clean = T0123_c[['ID', 'transmi__1', 'localidad', 'transmi__3', 'transmi__4', 'lat', 'lon', 'Q11', 'Q12', 'Q13', 'Q15', 'Q16', 'Q19', 'Q87c1', 'Q115', 'Q116']]
T0123_c_clean.head()
#len(T0123_c_clean)

,ID,transmi__1,localidad,transmi__3,transmi__4,lat,lon,Q11,Q12,Q13,Q15,Q16,Q19,Q87c1,Q115,Q116
0,ID15490250091142,1400,Ciudad Bolivar,Ciudad Bolivar,CL 69 I BIS SUR 18 M 45,4.553791,-74.153024,2,22,4,1,5,888,1,2,2
1,ID15210400150200,1232,Ciudad Bolivar,Ciudad Bolivar,CL 62 A SUR 18 Q 53,4.558492,-74.141770,1,50,1,1,4,888,888,2,1
2,ID15800280212297,2004,Ciudad Bolivar,Ciudad Bolivar,CL 73 C BIS SUR 27 55,4.544794,-74.161377,2,24,1,1,3,888,888,1,1
3,ID15800380210206,2022,Ciudad Bolivar,Ciudad Bolivar,KR 27 BIS 72 C 51 SUR,4.546388,-74.160283,2,73,1,1,3,888,888,1,1
4,ID15520940752473,1550,Ciudad Bolivar,Ciudad Bolivar,TV 18 P 69 A 33 SUR,4.554053,-74.149809,2,50,6,1,6,888,888,1,1


In [ ]:
# Save c_t_select df to csv to compare outseide of VSCode
#c_t_select.to_csv('/Users/katherinewu/Library/CloudStorage/GoogleDrive-kath.j.wu@gmail.com/My Drive/5. Coding/na-data-exploration/TrUST Data/c_t_select.csv', index=False)

In [ ]:
# Test merging T0_c merged df on T3 to see how many participants we can match in the final timeframe
T0123_ct = pd.merge(T0123_c_clean, c_t_select, left_on='transmi__4', right_on='Q2dir', how='inner')
print(T0123_ct.head())
print(f"Total number of rows: {len(T0123_ct)}") # sample size in sampling strategy is n = 598

                 ID  transmi__1       localidad      transmi__3  \
0  ID15490250091142        1400  Ciudad Bolivar  Ciudad Bolivar   
1  ID15530250061418        1613  Ciudad Bolivar  Ciudad Bolivar   
2  ID15530170031401        1607  Ciudad Bolivar  Ciudad Bolivar   
3  ID15800350451306        2012  Ciudad Bolivar  Ciudad Bolivar   
4  ID15490850211060        1444  Ciudad Bolivar  Ciudad Bolivar   

                transmi__4       lat        lon  Q11  Q12  Q13  ...  Q87c1  \
0  CL 69 I BIS SUR 18 M 45  4.553791 -74.153024    2   22    4  ...      1   
1    KR 18 N BIS A 66 D 11  4.554223 -74.146050    2   25    4  ...      1   
2      KR 18 Q 66 B 23 SUR  4.555241 -74.146830    2   29    4  ...    888   
3        KR 27 A 73 54 SUR  4.544984 -74.160630    1   56    6  ...    888   
4      CL 69 T SUR 18 M 63  4.550668 -74.152273    2   46    6  ...      2   

   Q115  Q116                    Q2dir  casas_ID     lat_x      lon_x  \
0     2     2  CL 69 I BIS SUR 18 M 45      1400  4.553

In [ ]:
c_t_select.dtypes

Q2dir          object
casas_ID        int64
lat_x         float64
lon_x         float64
trabajo_ID      int64
lat_y         float64
lon_y         float64
dtype: object

In [ ]:
T0123_c.dtypes

ID               object
transmi__1        int64
localidad        object
transmi__3       object
Q2dir            object
transmi__4       object
lat             float64
lon             float64
Q2dir            object
Q11               int64
Q12               int64
Q13               int64
Q15               int64
Q16               int64
Q19               int64
Q87c1             int64
Q115              int64
Q116              int64
T3_ID_INF        object
T3_ZONA_BASE     object
T2_ID            object
T2_localidad      int64
T1_ID            object
T1_localidad     object
T1_Q2dir          int64
dtype: object

In [ ]:
c_t.head()
len(c_t)

691

In [ ]:
# Save c_t df to csv to compare outseide of VSCode
#c_t.to_csv('/Users/katherinewu/Library/CloudStorage/GoogleDrive-kath.j.wu@gmail.com/My Drive/5. Coding/na-data-exploration/TrUST Data/c_t.csv', index=False)

In [ ]:
len(T0123_ct)

150

In [ ]:
T0123_ct.head()

,ID,transmi__1,localidad,transmi__3,transmi__4,lat,lon,Q11,Q12,Q13,...,Q87c1,Q115,Q116,Q2dir,casas_ID,lat_x,lon_x,trabajo_ID,lat_y,lon_y
0,ID15490250091142,1400,Ciudad Bolivar,Ciudad Bolivar,CL 69 I BIS SUR 18 M 45,4.553791,-74.153024,2,22,4,...,1,2,2,CL 69 I BIS SUR 18 M 45,1400,4.553791,-74.153024,1400,4.653890,-74.068424
1,ID15530250061418,1613,Ciudad Bolivar,Ciudad Bolivar,KR 18 N BIS A 66 D 11,4.554223,-74.146050,2,25,4,...,1,2,2,KR 18 N BIS A 66 D 11,1613,4.554223,-74.146050,1613,4.763943,-74.032297
2,ID15530170031401,1607,Ciudad Bolivar,Ciudad Bolivar,KR 18 Q 66 B 23 SUR,4.555241,-74.146830,2,29,4,...,888,2,2,KR 18 Q 66 B 23 SUR,1607,4.555241,-74.146830,1607,4.611747,-74.162797
3,ID15800350451306,2012,Ciudad Bolivar,Ciudad Bolivar,KR 27 A 73 54 SUR,4.544984,-74.160630,1,56,6,...,888,1,1,KR 27 A 73 54 SUR,2012,4.544984,-74.160630,2012,4.671269,-74.114723
4,ID15490850211060,1444,Ciudad Bolivar,Ciudad Bolivar,CL 69 T SUR 18 M 63,4.550668,-74.152273,2,46,6,...,2,2,2,CL 69 T SUR 18 M 63,1444,4.550668,-74.152273,1444,4.728757,-74.049864


In [ ]:
# Save T0123_ct df to csv to ingest into QGIS
#T0123_ct.to_csv('/Users/katherinewu/Library/CloudStorage/GoogleDrive-kath.j.wu@gmail.com/My Drive/5. Coding/na-data-exploration/TrUST Data/T0123_ct.csv', index=False)